#### Install yfinance

In [1]:
!pip install yfinance
!pip install prophet

     |████████████████████████████████| 6.3 MB 7.1 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23919 sha256=f4161f6226bca7af468c567ca74a723dba888896226e732889deccf27cd4a82d
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
     |████████████████████████████████| 65 kB 2.2 MB/s 
     |████████████████████████████████| 49 kB 3.6 MB/s 
     |████████████████████████████████| 179 kB 8.6 MB/s 
  Created wheel for prophet: filename=prophet-1.0.1-py3-none-any.whl size=6640260 sha256=a634448e7c0e00efc1bdf8a99278e181214cf0cdd76a3da58c7539279b151082
  Stored in directory: /root/.cache/pip/wheels/4e/a0/1a/02c9ec9e3e9de6bdbb3d769d11992a6926889d71567d6b9b67
Successfully built prophet
  Attempting uninstall: cmdstanpy
    Found exis

In [2]:
!pip install fbprophet

  Attempting uninstall: cmdstanpy
    Found existing installation: cmdstanpy 0.9.68
    Uninstalling cmdstanpy-0.9.68:
      Successfully uninstalled cmdstanpy-0.9.68
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
prophet 1.0.1 requires cmdstanpy==0.9.68, but you have cmdstanpy 0.9.5 which is incompatible.


# Forecasting Stocks from Yahoo Finance with Prophet

In [3]:
import yfinance as yf
import pandas as pd

import json
import plotly
import plotly.express as px

In [19]:
# berkshire hathaway inc. (b stock)
brk = yf.Ticker('VIVA3.SA')
hist = brk.history(period="max", auto_adjust=True)
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-10-11,24.017142,24.086329,23.443893,23.453777,7077400,0.0,0
2019-10-14,23.453776,23.542729,22.831110,23.236338,4214500,0.0,0
2019-10-15,23.325292,23.364826,22.979366,23.275873,1133500,0.0,0
2019-10-16,23.275872,23.819471,23.038666,23.819471,1946600,0.0,0
2019-10-17,23.720634,23.918306,23.572380,23.918306,904400,0.0,0
...,...,...,...,...,...,...,...
2021-07-22,31.840000,32.700001,31.820000,32.119999,981600,0.0,0
2021-07-23,32.259998,32.259998,31.530001,31.790001,496100,0.0,0
2021-07-26,31.780001,31.780001,31.219999,31.250000,447900,0.0,0


In [5]:
def gm(stock, period, interval):
    st = yf.Ticker(stock)

    # Create a line graph
    df = st.history(period=(period), interval=interval)
    df=df.reset_index()
    df.columns = ['Date-Time']+list(df.columns[1:])
    max = (df['Open'].max())
    min = (df['Open'].min())
    range = max - min
    margin = range * 0.05
    max = max + margin
    min = min - margin
    fig = px.area(df, x='Date-Time', y="Open",
        hover_data=("Open","Close","Volume"),
        range_y=(min,max), template="seaborn" )

    # Create a JSON representation of the graph
    graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
    return graphJSON
    #return fig.show()

In [6]:
gm("PETR3.SA", "1D", "1m")

'{"data": [{"customdata": [[28.030000686645508, 28.06999969482422, 0.0], [28.049999237060547, 28.100000381469727, 32100.0], [28.1200008392334, 28.190000534057617, 21500.0], [28.190000534057617, 28.170000076293945, 29000.0], [28.18000030517578, 28.170000076293945, 3600.0], [28.139999389648438, 28.149999618530273, 47500.0], [28.15999984741211, 28.18000030517578, 2800.0], [28.170000076293945, 28.15999984741211, 9800.0], [28.139999389648438, 28.1200008392334, 8600.0], [28.110000610351562, 28.100000381469727, 24500.0], [28.09000015258789, 28.100000381469727, 1800.0], [28.110000610351562, 28.1299991607666, 18000.0], [28.1299991607666, 28.1200008392334, 3500.0], [28.1200008392334, 28.1200008392334, 4800.0], [28.1200008392334, 28.149999618530273, 6200.0], [28.149999618530273, 28.149999618530273, 2800.0], [28.139999389648438, 28.139999389648438, 6600.0], [28.1200008392334, 28.1200008392334, 6900.0], [28.100000381469727, 28.059999465942383, 29600.0], [28.059999465942383, 28.059999465942383, 3310

In [20]:
df = pd.DataFrame()
df['ds'] = hist.index
df['y'] = hist['Close'].values
df.tail()

,ds,y
435,2021-07-22,32.119999
436,2021-07-23,31.790001
437,2021-07-26,31.250000
438,2021-07-27,30.490000
439,2021-07-28,31.200001


In [21]:
from fbprophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly#, plot_cross_validation_metric

In [22]:
m = Prophet(daily_seasonality=False)
m.fit(df)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


In [23]:
future = m.make_future_dataframe(180, freq='D')
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(3)

,ds,yhat,yhat_lower,yhat_upper
617,2022-01-22,42.186086,34.731526,50.551065
618,2022-01-23,42.241100,34.822120,50.300744
619,2022-01-24,42.244786,34.626734,50.852408


In [24]:
#m.plot(forecast, figsize=(14,7) );
plot_plotly(m, forecast)

In [12]:
#m.plot_components(forecast, figsize=(14,7));
plot_components_plotly(m, forecast, figsize=(1000, 200))

## Make it a Function

In [13]:
def make_forecast(ticker, periods, hist='max'):
    """
    forecast the given ticker (stock) period days into the future (from today)
    inputs
    ------
    > ticker
        >> ticker of stock to forecast
    > periods
        >> number of days into the future to forecast (from today's date)
    > hist
        >> amount of historical data to consider
            > default: max
            > options: 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max
    """
    # pull historical data from yahoo finance
    stock_data = yf.Ticker(ticker)

    hist_data = stock_data.history(hist, auto_adjust=True)

    # create new dataframe to hold dates (ds) & adjusted closing prices (y)
    df = pd.DataFrame()

    df['ds'] = hist_data.index.values
    df['y'] = hist_data['Close'].values

    # create a Prophet model from that data
    m = Prophet(daily_seasonality=False)
    m.fit(df)

    future = m.make_future_dataframe(periods, freq='D')

    forecast = m.predict(future)

    #m.plot(forecast)
    #plot_components_plotly(m, forecast)
    #plot_plotly(m, forecast)
    
    return plot_components_plotly(m, forecast)   

#### Make some Forecasts with that Function

In [14]:
make_forecast('PETR3.SA', 365)

In [15]:
df0 = df

In [17]:
df = df0.copy()
df = df[df['ds']>='2016-01-01']
df1=df.copy()
#df = df[:-estimated_days]
df_prophet = Prophet(changepoint_prior_scale=0.15,yearly_seasonality=True,daily_seasonality=True)
df_prophet.fit(df)
df_forecast = df_prophet.make_future_dataframe(periods= estimated_days*2, freq='D')
df_forecast = df_prophet.predict(df_forecast)
trace = go.Scatter(
    name = 'Actual price',
    mode = 'markers',
    x = list(df_forecast['ds']),
    y = list(df['y']),
    marker=dict(
        color='#FFBAD2',
        line=dict(width=1)
    )
)
trace1 = go.Scatter(
    name = 'trend',
    mode = 'lines',
    x = list(df_forecast['ds']),
    y = list(df_forecast['yhat']),
    marker=dict(
        color='red',
        line=dict(width=3)
    )
)
upper_band = go.Scatter(
    name = 'upper band',
    mode = 'lines',
    x = list(df_forecast['ds']),
    y = list(df_forecast['yhat_upper']),
    line= dict(color='#57b88f'),
    fill = 'tonexty'
)
lower_band = go.Scatter(
    name= 'lower band',
    mode = 'lines',
    x = list(df_forecast['ds']),
    y = list(df_forecast['yhat_lower']),
    line= dict(color='#1705ff')
)
tracex = go.Scatter(
    name = 'Actual price',
   mode = 'markers',
   x = list(df1['ds']),
   y = list(df1['y']),
   marker=dict(
      color='black',
      line=dict(width=2)
   )
)
data = [tracex, trace1, lower_band, upper_band, trace]

layout = dict(title='Bitcoin Price Estimation Using FbProphet',
             xaxis=dict(title = 'Dates', ticklen=2, zeroline=True))

figure=dict(data=data,layout=layout)
plt.savefig('btc03.png')
py.offline.iplot(figure)

NameError: ignored